# Market Basket Analyse/ Assoziationsregeln

Das Lernen von Assoziationsregeln (Association rule learning) ist ein Verfahren, um aus Datenmengen automatisiert Regeln oder Abhängigkeiten zu lernen.

Ein sehr bekanntes Beispiel hierfür ist die Anzeige von "Kunden, die dies kauften, kauften auch ...." auf Amazon. Das wird auch Market Basket Analysis genannt.

Wir schauen uns hier anhand eines beispielhaften Datensatzes eines Retail-Laden das im Detail an. Den Datensatz können Sie inkl. kurzer Erklärung [hier](https://archive.ics.uci.edu/dataset/352/online+retail) finden.

## Daten organisieren

In [1]:
# Importieren der Standard-Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Load the file into pandas
retail_df = pd.read_excel("http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx")

## Datenaufbereitung

In [3]:
retail_df.describe(include="all")

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
count,541909.0,541909,540455,541909.000000,541909,541909.000000,406829.000000,541909
unique,25900.0,4070,4223,NaN,NaN,NaN,NaN,38
top,573585.0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,NaN,NaN,NaN,United Kingdom
freq,1114.0,2313,2369,NaN,NaN,NaN,NaN,495478
mean,NaN,NaN,NaN,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570,NaN
min,NaN,NaN,NaN,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000,NaN
25%,NaN,NaN,NaN,1.000000,2011-03-28 11:34:00,1.250000,13953.000000,NaN
50%,NaN,NaN,NaN,3.000000,2011-07-19 17:17:00,2.080000,15152.000000,NaN
75%,NaN,NaN,NaN,10.000000,2011-10-19 11:27:00,4.130000,16791.000000,NaN
max,NaN,NaN,NaN,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000,NaN


In [4]:
retail_df.isna().sum()

,0
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0


Wir sehen daß uns sehr viele Kundennummern fehlen. Deshalb löschen wir diese Spalte.

In [5]:
modified_retail_df = retail_df
modified_retail_df['Description'] = modified_retail_df['Description'].str.strip()

In der Dokumentation zum Datensatz steht daß alle Invoice-Nummern, die mit C anfangen, abgebrochene Bestellungen sind. Deshalb löschen wir die jetzt.

In [6]:
modified_retail_df['InvoiceNo'].str.contains('C').sum()

9288

Es gibt 9288 Bestellungen, die abgebrochen wurden

In [7]:
# Umwandeln der Spalte in Zeichenkette
modified_retail_df.dropna(axis = 0, subset =['InvoiceNo'], inplace = True)
modified_retail_df["InvoiceNo"] = modified_retail_df["InvoiceNo"].astype('str')

# damit wir nach Spalten mit C für cancelled filtern können
modified_retail_df = modified_retail_df[~modified_retail_df["InvoiceNo"].str.contains("C")]

In [8]:
modified_retail_df.isna().sum()

,0
InvoiceNo,0
StockCode,0
Description,1455
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,134697
Country,0


Wir haben noch 1454 Zeilen ohne Artikelbeschreibung. Diese löschen wir; es hilft bei Regeln nicht viel, wenn wir den Artikelnamen nicht kennen.

In [9]:
modified_retail_df = modified_retail_df.dropna()

In [10]:
modified_retail_df.isna().sum()

,0
InvoiceNo,0
StockCode,0
Description,0
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,0
Country,0


In [11]:
modified_retail_df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


Wir sehen daß die ersten 4 Zeilen die gleiche Invoice-Nummer haben, also eine gleiche Bestellung sind.
Für eine Basketanalyse müssen wir jetzt noch eine Art virtuellen Einkaufskorb erstellen. Dazu verwenden wir den groupby-Befehl.

Dieser sagt, daß wir alle Zeilen, die die gleiche Invoice-Nummer haben, gruppieren in eine neue Zeile in einer Liste, gleiche Elemente aufaddieren.

Wir schauen uns nun an, wie sich Kaufaktionen in verschiedenen Ländern gestalten.

In [12]:
modified_retail_df['Country'].value_counts()

,count
Country,
United Kingdom,354345
Germany,9042
France,8342
EIRE,7238
Spain,2485
Netherlands,2363
Belgium,2031
Switzerland,1842
Portugal,1462


UK und Deutschland sind die häufigsten, also schauen wir uns nur die beiden an.

In [13]:
# Befehl entnommen: https://www.geeksforgeeks.org/implementing-apriori-algorithm-in-python/

basket_UK = (modified_retail_df[modified_retail_df['Country'] =="United Kingdom"] # filtere auf alle Käufe die in England getan wurden
          .groupby(['InvoiceNo', 'Description'])['Quantity']  # gruppeiere anhand Bestellnummer und Beschreibung
          .sum()  # summiere die einzeln gefunden Elemente auf
          .unstack()
          .reset_index() # erstelle einen neuen Index
          .fillna(0)  # fülle leere Zeilen mit 0 auf
          .set_index('InvoiceNo'))  # Zeilenindex ist auf Invoice.Nr.

basket_GER = (modified_retail_df[modified_retail_df['Country'] =="Germany"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum()
          .unstack()
          .reset_index()
          .fillna(0)
          .set_index('InvoiceNo'))

basket_FRA = (modified_retail_df[modified_retail_df['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum()
          .unstack()
          .reset_index()
          .fillna(0)
          .set_index('InvoiceNo'))



Für den Assoziationsregelalgorithmus müssen wir noch eine Sache machen: der Algorithmus verlangt daß wir statt der Anzahl an gekauften Items einen False-Wert haben, falls nichts gekauft wurde und True sonst.

In [14]:
def encode_values(x):
    if(x <= 0):
        return 0
    if(x >= 1):
        return 1

# Encoding the datasets
basket_UK = basket_UK.applymap(encode_values)
basket_GER = basket_GER.applymap(encode_values)
basket_FRA = basket_FRA.applymap(encode_values)

/tmp/ipython-input-3347687889.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_UK = basket_UK.applymap(encode_values)
/tmp/ipython-input-3347687889.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_GER = basket_GER.applymap(encode_values)
/tmp/ipython-input-3347687889.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_FRA = basket_FRA.applymap(encode_values)


## Berechnen von Assoziationsregeln

Wir laden erst die notwendigen Bibliotheken

In [15]:
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import apriori, association_rules

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [16]:
# Model erstellen für Deutschland.
frq_items_GER = apriori(basket_GER,
                       min_support = 0.1, # mindestens 10% Support muss eine Regel haben
                       use_colnames = True)

# speichern der Regeln
rules_GER = association_rules(frq_items_GER,
                              metric ="lift",
                              min_threshold = 1)
# sortieren der Regeln
rules_GER = rules_GER.sort_values(['confidence', 'lift'],
                                  ascending =[False, False])

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [17]:
frq_items_GER

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

,support,itemsets
0,0.102845,(6 RIBBONS RUSTIC CHARM)
1,0.100656,(JUMBO BAG WOODLAND ANIMALS)
2,0.115974,(PLASTERS IN TIN CIRCUS PARADE)
3,0.107221,(PLASTERS IN TIN SPACEBOY)
4,0.137856,(PLASTERS IN TIN WOODLAND ANIMALS)
5,0.818381,(POSTAGE)
6,0.137856,(REGENCY CAKESTAND 3 TIER)
7,0.157549,(ROUND SNACK BOXES SET OF 4 FRUITS)
8,0.245077,(ROUND SNACK BOXES SET OF4 WOODLAND)
9,0.102845,(SPACEBOY LUNCH BOX)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [18]:
rules_GER

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
8,(ROUND SNACK BOXES SET OF 4 FRUITS),(POSTAGE),0.157549,0.818381,0.150985,0.958333,1.171012,1.0,0.022049,4.358862,0.173348,0.183024,0.770582,0.571413
16,"(ROUND SNACK BOXES SET OF 4 FRUITS, ROUND SNAC...",(POSTAGE),0.131291,0.818381,0.124726,0.950000,1.160829,1.0,0.017280,3.632385,0.159486,0.151194,0.724699,0.551203
2,(PLASTERS IN TIN SPACEBOY),(POSTAGE),0.107221,0.818381,0.100656,0.938776,1.147113,1.0,0.012909,2.966448,0.143649,0.122016,0.662896,0.530885
10,(ROUND SNACK BOXES SET OF4 WOODLAND),(POSTAGE),0.245077,0.818381,0.225383,0.919643,1.123735,1.0,0.024817,2.260151,0.145856,0.268930,0.557552,0.597522
12,(WOODLAND CHARLOTTE BAG),(POSTAGE),0.126915,0.818381,0.115974,0.913793,1.116587,1.0,0.012109,2.106783,0.119591,0.139842,0.525343,0.527752
6,(REGENCY CAKESTAND 3 TIER),(POSTAGE),0.137856,0.818381,0.120350,0.873016,1.066760,1.0,0.007532,1.430252,0.072589,0.143979,0.300822,0.510037
0,(PLASTERS IN TIN CIRCUS PARADE),(POSTAGE),0.115974,0.818381,0.100656,0.867925,1.060539,1.0,0.005746,1.375117,0.064572,0.120735,0.272789,0.495460
4,(PLASTERS IN TIN WOODLAND ANIMALS),(POSTAGE),0.137856,0.818381,0.118162,0.857143,1.047364,1.0,0.005344,1.271335,0.052453,0.140992,0.213425,0.500764
14,(ROUND SNACK BOXES SET OF 4 FRUITS),(ROUND SNACK BOXES SET OF4 WOODLAND),0.157549,0.245077,0.131291,0.833333,3.400298,1.0,0.092679,4.529540,0.837922,0.483871,0.779227,0.684524
17,"(ROUND SNACK BOXES SET OF 4 FRUITS, POSTAGE)",(ROUND SNACK BOXES SET OF4 WOODLAND),0.150985,0.245077,0.124726,0.826087,3.370730,1.0,0.087724,4.340810,0.828405,0.459677,0.769628,0.667508


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [19]:
frq_items = apriori(basket_UK, min_support = 0.1, use_colnames = True)
rules_UK = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules_UK = rules_UK.sort_values(['confidence', 'lift'], ascending =[False, False])

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [20]:
rules_UK;

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [21]:
frq_items = apriori(basket_FRA, min_support = 0.1, use_colnames = True)
rules_FRA = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules_FRA = rules_FRA.sort_values(['confidence', 'lift'], ascending =[False, False])

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [22]:
rules_FRA

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
49,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER PLATES),0.102828,0.128535,0.100257,0.975000,7.585500,1.0,0.087040,34.858612,0.967673,0.764706,0.971313,0.877500
48,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER CUPS),0.102828,0.138817,0.100257,0.975000,7.023611,1.0,0.085983,34.447301,0.955918,0.709091,0.970970,0.848611
41,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.128535,0.138817,0.123393,0.960000,6.915556,1.0,0.105550,21.529563,0.981563,0.857143,0.953552,0.924444
44,"(SET/6 RED SPOTTY PAPER PLATES, POSTAGE)",(SET/6 RED SPOTTY PAPER CUPS),0.107969,0.138817,0.102828,0.952381,6.860670,1.0,0.087840,18.084833,0.957637,0.714286,0.944705,0.846561
34,(STRAWBERRY LUNCH BOX WITH CUTLERY),(POSTAGE),0.123393,0.771208,0.115681,0.937500,1.215625,1.0,0.020519,3.660668,0.202346,0.148515,0.726826,0.543750
26,(ROUND SNACK BOXES SET OF4 WOODLAND),(POSTAGE),0.159383,0.771208,0.149100,0.935484,1.213011,1.0,0.026183,3.546272,0.208900,0.190789,0.718014,0.564409
18,(RABBIT NIGHT LIGHT),(POSTAGE),0.187661,0.771208,0.167095,0.890411,1.154566,1.0,0.022370,2.087725,0.164800,0.211039,0.521010,0.553539
40,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.138817,0.128535,0.123393,0.888889,6.915556,1.0,0.105550,7.843188,0.993284,0.857143,0.872501,0.924444
22,(RED TOADSTOOL LED NIGHT LIGHT),(POSTAGE),0.179949,0.771208,0.159383,0.885714,1.148476,1.0,0.020605,2.001928,0.157650,0.201299,0.500482,0.546190
10,(PLASTERS IN TIN CIRCUS PARADE),(POSTAGE),0.169666,0.771208,0.149100,0.878788,1.139495,1.0,0.018253,1.887532,0.147432,0.188312,0.470208,0.536061


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag